In [3]:
import nbimporter
import read_report_cobol as cobol_reader # type: ignore
import read_report_java as java_reader # type: ignore

cobol = cobol_reader.get_cobol_data_from("RHPP05LA.PDF")
java = java_reader.get_java_data_from("RHPP05LA - GERAL.pdf")